无数据增强，但有图像缩小

In [90]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout

In [91]:
# 缩放标签坐标
def scale_labels(label_data, scale_percent):
    label_data[['LE_left', 'LE_top', 'LE_right', 'LE_bottom',
                'RE_left', 'RE_top', 'RE_right', 'RE_bottom',
                'lx', 'ly', 'rx', 'ry']] = label_data[['LE_left', 'LE_top', 'LE_right', 'LE_bottom',
                                                        'RE_left', 'RE_top', 'RE_right', 'RE_bottom',
                                                        'lx', 'ly', 'rx', 'ry']].astype(float)
    label_data[['LE_left', 'LE_top','LE_right', 'LE_bottom','RE_left', 'RE_top','RE_right','RE_bottom', 'lx', 'ly','rx','ry']] *= scale_percent
    return label_data

# 缩放和保存标签
def scale_and_save_labels(label_folder, output_label_folder, scale_percent):
    if not os.path.exists(output_label_folder):
        os.makedirs(output_label_folder)  #如果文件夹不存在，就创建该文件夹
    for label_file in os.listdir(label_folder):
        if label_file.endswith('.csv'):
            label_path = os.path.join(label_folder, label_file)
            output_label_path = os.path.join(output_label_folder, label_file)
            label_data = pd.read_csv(label_path)  #读取标签文件的数据
            scaled_label_data = scale_labels(label_data, scale_percent)  #将数据进行缩放
            scaled_label_data.to_csv(output_label_path, index=False)  #存储到csv文件里

In [92]:
# 加载图像和标签
def load_images_and_labels(image_folder, label_folder, target_size=(299, 299)):
    images = []
    labels = []
    #遍历标签文件
    for label_file in os.listdir(label_folder):
        if label_file.endswith('.csv'):
            participant_video = label_file[:-4]  ## 去除扩展名".csv"
            participant, video = participant_video.split('_')   ## 将csv的文件名如'001_0'划分为'001'和'0'

            # 构建图像文件夹路径
            image_folder_path = os.path.join(image_folder, participant, video)  ## 即获得'D:/Bristol/finalCoursework/dataset/ECL_Images_Test/001/0'

            if not os.path.isdir(image_folder_path):
                continue  #确保是有效目录
            label_path = os.path.join(label_folder, label_file) # 即'D:/Bristol/finalCoursework/dataset/ECL_Labels_Test/001_0.csv'
            label_data = pd.read_csv(label_path)
            for index, row in label_data.iterrows():
                image_name = row['filename']
                #image_folder_path = 'D:/Bristol/finalCoursework/dataset/ECL_Images_Test/'
                image_path = os.path.join(image_folder, image_name)
                print("文件path为：" + image_path)

                if not os.path.exists(image_path):
                    print(f"Image file does not exist: {image_path}")
                    continue
                
                image = cv2.imread(image_path)
                if image is None:
                    continue  #确保图像读取成功

                image = cv2.resize(image, target_size)

                #将处理后的图像存储到本地
                sanitized_image_name = image_name.replace(':', '_').replace(' ', '_') # 处理文件名中的非法字符
                output_image_path = os.path.join(output_image_folder, participant, video, sanitized_image_name)

                # 确保保存图像的目录存在
                os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

                cv2.imwrite(output_image_path, image)
                if os.path.exists(output_image_path):
                    print(f"Saved processed image: {output_image_path}")
                else:
                    print(f"Failed to save processed image: {output_image_path}")

                label = row[['LE_left', 'LE_top', 'LE_right', 'LE_bottom',
                             'RE_left', 'RE_top', 'RE_right', 'RE_bottom',
                             'lx', 'ly', 'rx', 'ry']].values.astype('float32')  # 确保标签数据为 float32 类型
                images.append(image)
                labels.append(label)

                #归一化
                image = image.astype('float32')  # 转换图像数据类型为 float32
                image = preprocess_input(image)  # 使用预处理函数进行归一化

        print(f"Loaded {len(images)} images and {len(labels)} labels.")

    return np.array(images), np.array(labels)

#缩放标签并保存
label_folder = 'D:/Bristol/finalCoursework/dataset/ECL_Labels_Test'
output_label_folder = 'D:/Bristol/finalCoursework/dataset/ECL_Labels_Scaled'
scale_percent = 299/960  # 缩放比例为图片的缩放比例
scale_and_save_labels(label_folder, output_label_folder, scale_percent)

# 加载图像和标签
image_folder = 'D:/Bristol/finalCoursework/dataset/ECL_Images_Test'
output_image_folder = 'D:/Bristol/finalCoursework/dataset/Processed_Images'
#labels_folder = output_label_folder  # 使用缩放后的标签文件夹
label_folder = 'D:/Bristol/finalCoursework/dataset/ECL_Labels_Scaled'
#print("label_folder", label_folder)
#label_folder = 'D:/Bristol/finalCoursework/dataset/ECL_Labels_Scaled'
images, labels = load_images_and_labels(image_folder, label_folder, target_size=(299, 299))
print(f"Loaded {images.shape[0]} images with shape {images.shape[1:]} and {labels.shape[0]} labels.")

文件path为：D:/Bristol/finalCoursework/dataset/ECL_Images_Test\001/0/2022-09-26 08_07_22.792702-37704.jpg
Saved processed image: D:/Bristol/finalCoursework/dataset/Processed_Images\001\0\001/0/2022-09-26_08_07_22.792702-37704.jpg
文件path为：D:/Bristol/finalCoursework/dataset/ECL_Images_Test\001/0/2022-09-26 08_07_23.101953-37752.jpg
Saved processed image: D:/Bristol/finalCoursework/dataset/Processed_Images\001\0\001/0/2022-09-26_08_07_23.101953-37752.jpg
文件path为：D:/Bristol/finalCoursework/dataset/ECL_Images_Test\001/0/2022-09-26 08_07_22.818864-37708.jpg
Saved processed image: D:/Bristol/finalCoursework/dataset/Processed_Images\001\0\001/0/2022-09-26_08_07_22.818864-37708.jpg
文件path为：D:/Bristol/finalCoursework/dataset/ECL_Images_Test\001/0/2022-09-26 08_07_23.225207-37772.jpg
Saved processed image: D:/Bristol/finalCoursework/dataset/Processed_Images\001\0\001/0/2022-09-26_08_07_23.225207-37772.jpg
文件path为：D:/Bristol/finalCoursework/dataset/ECL_Images_Test\001/0/2022-09-26 08_07_23.126469-3775

In [93]:
# 检查数据形状
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (161, 299, 299, 3)
Labels shape: (161, 12)


In [94]:
print(labels)

[[ 40.80104  127.697914 116.796875 ... 146.38542  238.57709  150.43437 ]
 [ 50.767708 149.18854  118.04271  ... 165.38437  230.47917  167.25313 ]
 [ 40.489582 127.697914 116.17396  ... 146.69687  237.64272  151.36874 ]
 ...
 [ 33.94896  120.84583  110.56771  ... 139.22188  234.83958  144.20522 ]
 [ 34.260418 120.53438  110.56771  ... 138.59895  234.52812  143.89375 ]
 [ 33.94896  118.97708  109.32188  ... 136.41875  234.83958  142.025   ]]


In [95]:
# # 定义模型
# def create_model(input_shape):
#     base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape) #加载预训练的 InceptionResNetV2 模型，使用 ImageNet 预训练的权重。
#     x = base_model.output  # 获取基础模型的输出，即 InceptionResNetV2 的最后一层输出。
#     x = GlobalAveragePooling2D()(x)
#     # 在基础模型的输出后添加全局平均池化层。
#     # GlobalAveragePooling2D() 会对每个特征图的空间维度进行全局平均池化，将其转换为单个值。
#     # 这将减少参数的数量并有助于防止过拟合。
#     x = Dense(512, activation='relu')(x)
#     # 添加一个全连接层（Dense 层），有 512 个神经元，使用 ReLU 激活函数。
#     # 这一层将学习到更加抽象和高层次的特征。
#     predictions = Dense(12, activation='linear')(x)
#     # 添加一个全连接层（Dense 层），有 12 个神经元，使用线性激活函数。
#     # 这一层的输出是 12 个值，适合用于回归任务，例如输出多个坐标值或其他连续值。
#     model = Model(inputs=base_model.input, outputs=predictions)
#     # 创建一个 Keras 模型，指定输入和输出。
#     # inputs=base_model.input 表示模型的输入是基础模型的输入。
#     # outputs=predictions 表示模型的输出是我们在基础模型顶部添加的自定义层的输出。
#     return model

def create_model(input_shape):
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)  # 添加Dropout层
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # 添加Dropout层
    predictions = Dense(12, activation='linear', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)  # 添加L2正则化
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [96]:
# 计算误差小于0.1条件下的准确率
def calculate_accuracy(predictions, labels, threshold=0.1):
    # errors = np.abs(predictions - labels)
    # accurate_predictions = np.all(errors < threshold, axis=1)
    # accuracy = np.mean(accurate_predictions)
    # return accuracy
    # 提取左眼和右眼的坐标
    pred_left_eye = predictions[:, 8:10]
    pred_right_eye = predictions[:, 10:12]
    true_left_eye = labels[:, 8:10]
    true_right_eye = labels[:, 10:12]
    print("pred_left_eye:", pred_left_eye)
    print("pred_right_eye",pred_right_eye)
    print("true_left_eye",true_left_eye)
    print("true_right_eye",true_right_eye)
    
    # 计算欧几里得距离
    left_eye_distances = np.linalg.norm(pred_left_eye - true_left_eye, axis=1)
    right_eye_distances = np.linalg.norm(pred_right_eye - true_right_eye, axis=1)
    
    # 判断距离是否在阈值范围内
    left_eye_accurate = left_eye_distances < threshold
    right_eye_accurate = right_eye_distances < threshold
    
    # 计算准确率
    accurate_predictions = np.logical_and(left_eye_accurate, right_eye_accurate)
    accuracy = np.mean(accurate_predictions)
    
    return accuracy

In [97]:
class AccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, threshold=0.1):
        self.validation_data = validation_data
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_images, val_labels = self.validation_data
        predictions = self.model.predict(val_images)
        accuracy = calculate_accuracy(predictions, val_labels, threshold=self.threshold)
        print(f'Epoch {epoch+1}: accuracy = {accuracy:.2f}')

In [98]:
# # 分割训练和测试数据
# from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# 创建模型
model = create_model(input_shape=(299, 299, 3))

# 第一阶段：冻结预训练模型权重
for layer in model.layers[:-2]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error')

# 训练模型（第一阶段）
callbacks = [
    ModelCheckpoint('model_stage2.keras', save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True),
    AccuracyCallback(validation_data=(test_images, test_labels), threshold=0.1)
]

model.fit(train_images, train_labels, validation_split=0.2, epochs=10, callbacks=callbacks)

# 第二阶段：解冻整个模型的权重
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='mean_squared_error')

# 训练模型（第二阶段）
callbacks = [
    ModelCheckpoint('model_stage2.keras', save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True),
    AccuracyCallback(validation_data=(test_images, test_labels), threshold=0.1)
]

model.fit(train_images, train_labels, validation_split=0.2, epochs=10, callbacks=callbacks)

# 预测和计算准确率
predictions = model.predict(test_images)
accuracy = calculate_accuracy(predictions, test_labels)

print(f"Model accuracy with error threshold of 0.1: {accuracy:.2f}")

# 保存最终模型
model.save('final_eye_localization_model.keras')

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 6s/step  loss: 37250.01
pred_left_eye: [[70.38052  15.431087]
 [70.178406 15.712279]
 [63.387432 14.244707]
 [68.00074  15.326922]
 [64.20763  14.368946]
 [81.349144 17.680977]
 [65.880394 14.982147]
 [63.616882 14.190997]
 [68.74768  15.231302]
 [69.39196  15.642603]
 [81.27019  18.05075 ]
 [82.10834  17.767029]
 [71.998474 16.603655]
 [68.86603  15.039317]
 [68.629166 15.349121]
 [65.1475   14.70086 ]
 [69.73681  15.707503]
 [64.76058  14.63131 ]
 [63.669636 14.213042]
 [67.91591  15.030636]
 [70.26753  15.751789]
 [84.68539  18.07993 ]
 [69.43419  15.864693]
 [82.97388  17.344707]
 [69.14984  15.388578]
 [67.077614 14.96964 ]
 [64.764656 14.428052]
 [64.73955  14.574203]
 [70.1832   15.835052]
 [68.440506 15.349236]
 [65.59358  14.779393]
 [64.48299  14.314318]
 [66.37219  14.817126]]
pred_right_eye [[-2.290679   42.845207  ]
 [-1.4367951  43.525784  ]
 [-2.275508   36.84834   ]
 [-1.7627476  40.861465  ]
 [-2.01612    37.45197   ]
 [ 1.721209

KeyboardInterrupt: 